In [ ]:
#load the dir

from google.colab import drive
drive.mount('/content/gdrive/')
#%cd /content/gdrive/MyDrive/
#!ls
#!git clone https://github.com/JonasGeiping/invertinggradients.git
%cd /content/gdrive/MyDrive/MTD/experiments/
!ls

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/MyDrive/MTD/experiments
 example   nvdcve-1.1-2021.json  'small beta'


In [ ]:
#load NVD data

import json

# Opening JSON file
fObj = open('nvdcve-1.1-2021.json',)

# It returns JSON object as dictionary
ogdata = json.load(fObj)

# Iterating through the json list
for i in ogdata['CVE_Items']:
    print(i["cve"]["CVE_data_meta"]["ID"])

# Closing file
fObj.close()

Streaming output truncated to the last 5000 lines.
CVE-2021-39046
CVE-2021-39047
CVE-2021-39048
CVE-2021-39049
CVE-2021-39050
CVE-2021-39051
CVE-2021-39052
CVE-2021-39053
CVE-2021-39054
CVE-2021-39055
CVE-2021-39056
CVE-2021-39057
CVE-2021-39058
CVE-2021-39059
CVE-2021-3906
CVE-2021-39063
CVE-2021-39064
CVE-2021-39065
CVE-2021-39066
CVE-2021-39068
CVE-2021-3907
CVE-2021-39070
CVE-2021-39072
CVE-2021-39074
CVE-2021-39076
CVE-2021-39078
CVE-2021-39079
CVE-2021-3908
CVE-2021-39080
CVE-2021-39082
CVE-2021-3909
CVE-2021-3910
CVE-2021-39109
CVE-2021-3911
CVE-2021-39111
CVE-2021-39112
CVE-2021-39113
CVE-2021-39114
CVE-2021-39115
CVE-2021-39116
CVE-2021-39117
CVE-2021-39118
CVE-2021-39119
CVE-2021-3912
CVE-2021-39121
CVE-2021-39122
CVE-2021-39123
CVE-2021-39124
CVE-2021-39125
CVE-2021-39126
CVE-2021-39127
CVE-2021-39128
CVE-2021-39131
CVE-2021-39132
CVE-2021-39133
CVE-2021-39134
CVE-2021-39135
CVE-2021-39136
CVE-2021-39137
CVE-2021-39138
CVE-2021-39139
CVE-2021-39140
CVE-2021-39141
CVE-2021-39

In [ ]:
import math
import copy

import os
import random

import numpy as np
import gym
from gym import spaces
#from gym.utils import seeding
import time

!pip install stable_baselines3
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class Attacker_Env(gym.Env):

    def __init__(self):
        
        self.rnd=0

        high = 1
        low = 0
        self.action_space = spaces.Box(
            low=low,
            high=high,
            shape=(num_config,),
            dtype=np.float32
        )

        #self.observation_space=spaces.MultiDiscrete([num_config, max_tau, max_tau])
        self.observation_space=spaces.MultiDiscrete([num_config, 2])
        


    def step(self, action):
        
        
        self.rnd+=1
        #print(action)
        
        #att_action=action+1
        att_action=action/np.sum(action)
        #print(action)
        #att_config=np.argmax(action*q*g)
        
        if self.training_length==0: def_action=np.array([1/num_config]*num_config)
        else: 
            def_action, _ = self.model.predict(self.obs)
            def_action=att_action/np.sum(att_action)

        att_config=np.random.choice(num_config, p=att_action)
        
        old_config=self.config
        self.config = np.random.choice(num_config, p=def_action)
        if self.config==old_config: self.tau+=1
        else: 
            self.tau=1
            self.state=0
            self.tau_a=0
        
        if att_config==self.config and self.state==0: self.state=np.random.choice([0,1], p=[1-q[self.config], q[self.config]])
        if self.state==1: self.tau_a+=1

        reward=self.state*r1[self.config]
        #if math.isinf(reward) or math.isnan(reward):
            #print("abnormal reward")
            #reward = 0
        
        done=False
        if self.rnd==max_rnd: done=True

        #self.obs=np.array([self.config,self.tau, self.tau_a])
        self.obs=np.array([self.config,self.state])
        
        return self.obs, reward, done, {}

    def reset(self):

        self.rnd=0
        self.config=np.random.choice(num_config, p=initial_distribution)
        self.state=0
        self.tau=0
        self.tau_a=0
        
        self.training_length=110000
        while os.path.exists("ALTdefender_TD3_"+str(self.training_length)+".zip")==False:
            self.training_length-=10000
            #print(self.training_length)
            if self.training_length==0: break
            if os.path.exists("ALTdefender_TD3_"+str(self.training_length)+".zip"):
                self.model = TD3.load("ALTdefender_TD3_"+str(self.training_length)+".zip")
        
        #self.obs=np.array([self.config,self.tau, self.tau_a])
        self.obs=np.array([self.config,self.state])
        
        return self.obs

In [ ]:
#test attacker's env

env = Attacker_Env()

print(env.observation_space)
print(env.action_space)

#num_states = env.observation_space.shape[0]
#print("Size of State Space ->  {}".format(num_states))
num_actions = env.action_space.shape[0]
print("Size of Action Space ->  {}".format(num_actions))
#print(env.action_space.shape[-1])

upper_bound = env.action_space.high[0]
lower_bound = env.action_space.low[0]
print("Max Value of Action ->  {}".format(upper_bound))
print("Min Value of Action ->  {}".format(lower_bound))


#torch.autograd.set_detect_anomaly(True)

start = time.time()
env.reset()
#mid = time.time()
done=False
while done==False:
  action=np.random.uniform(low=0.0, high=1.0, size=4)
  state, reward, done, _ = env.step(action)
  #print(reward)
end = time.time()
print(end-start)
#print(mid-start)
#print(end - mid)
#print((end-mid)/20)

#torch.autograd.set_detect_anomaly(False)

MultiDiscrete([4 2])
Box([0. 0. 0. 0.], [1. 1. 1. 1.], (4,), float32)
Size of Action Space ->  4
Max Value of Action ->  1.0
Min Value of Action ->  0.0
0.013000249862670898


In [ ]:
num_config=4
max_rnd=100
#max_tau=max_rnd
initial_distribution=np.array([1,0,0,0])

M=np.array([[0,1,2,3],
            [2,0,1,2],
            [2,3,0,1],
            [3,2,1,0]])


In [ ]:
class Defender_Env(gym.Env):

    def __init__(self,r,q):
        
        self.rnd=0
        self.r=r
        self.q=q

        high = 1
        low = 0
        self.action_space = spaces.Box(
            low=low,
            high=high,
            shape=(num_config,),
            dtype=np.float32
        )

        #self.observation_space=spaces.MultiDiscrete([num_config, max_tau, max_tau])
        #self.observation_space=spaces.MultiDiscrete([num_config, 2])
        self.observation_space=spaces.Discrete(num_config)
        


    def step(self, action):
        
        
        self.rnd+=1
        #print(action)
        
        def_action=action/np.sum(action)
        #print(action)
        
        #att_config=np.argmax(def_action*q*r)
        
        #if self.training_length==0:
            #print('random attack!') 
            #att_action=np.array([1/num_config]*num_config)
        #else: 
            #att_action, _ = self.model.predict(self.obs)
            #att_action=att_action/np.sum(att_action)
        
        if self.mod==1:  
            att_action=np.array([1/num_config]*num_config)
            att_config=np.random.choice(num_config, p=att_action)
        
        else: att_config=np.argmax(def_action*self.q*self.r)
        
        
        old_config=self.config
        self.config = np.random.choice(num_config, p=def_action)
        #if self.config==old_config: self.tau+=1
        #else: 
            #self.tau=1
            #self.state=0
            #self.tau_a=0
        self.state=0
        if att_config==self.config and self.state==0: self.state=np.random.choice([0,1], p=[1-self.q[self.config], self.q[self.config]])
        #if self.state==1: self.tau_a+=1

        #if self.config==old_config: reward=-self.state*self.r[self.config]
        #else: reward=-M[old_config, self.config]
        reward=-self.state*self.r[self.config]-M[old_config, self.config]
        
        
        #print(self.state)

        
        done=False
        if self.rnd==max_rnd: done=True

        #self.obs=np.array([self.config,self.tau, self.tau_a])
        #self.obs=np.array([self.config,self.state])
        self.obs=self.config
        
        return self.obs, reward, done, {}

    def reset(self):

        self.rnd=0
        self.config=np.random.choice(num_config, p=initial_distribution)
        #self.state=0
        #self.tau=0
        #self.tau_a=0
        
        #self.training_length=110000
        #while os.path.exists("ALTattacker_TD3_"+str(self.training_length)+".zip")==False:
            #self.training_length-=10000
            #print(self.training_length)
            #if self.training_length==0: break
            #if os.path.exists("ALTattacker_TD3_"+str(self.training_length)+".zip"):
                #self.model = TD3.load("ALTattacker_TD3_"+str(self.training_length)+".zip")
        
        self.mod=np.random.choice([0,1])
        #if self.mod<=2: self.r=r1
        #else: self.r=r2
        

        #self.obs=np.array([self.config,self.tau, self.tau_a])
        #self.obs=np.array([self.config,self.state])
        self.obs=self.config

        return self.obs

In [ ]:
#test defender's env

r=10*np.random.randint(11, size=num_config)
print(r)
q=0.1*np.random.randint(11, size=num_config)
print(q)
env = Defender_Env(r,q)
#env = Defender_Env()
print(env.observation_space)
print(env.action_space)

#num_states = env.observation_space.shape[0]
#print("Size of State Space ->  {}".format(num_states))
num_actions = env.action_space.shape[0]
print("Size of Action Space ->  {}".format(num_actions))
#print(env.action_space.shape[-1])

upper_bound = env.action_space.high[0]
lower_bound = env.action_space.low[0]
print("Max Value of Action ->  {}".format(upper_bound))
print("Min Value of Action ->  {}".format(lower_bound))


#torch.autograd.set_detect_anomaly(True)

start = time.time()
env.reset()
#mid = time.time()
done=False
while done==False:
  action=np.random.uniform(low=0.0, high=1.0, size=4)
  state, reward, done, _ = env.step(action)
  #print(reward)
end = time.time()
print(end-start)
#print(mid-start)
#print(end - mid)
#print((end-mid)/20)

#torch.autograd.set_detect_anomaly(False)

[70 20 60 60]
[0.1 0.3 0.1 0.3]
Discrete(4)
Box([0. 0. 0. 0.], [1. 1. 1. 1.], (4,), float32)
Size of Action Space ->  4
Max Value of Action ->  1.0
Min Value of Action ->  0.0
0.01880669593811035


In [ ]:
#empty buffer

env1 = Defender_Env()
#env2 = Attacker_Env()

# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model1 = TD3("MlpPolicy", env1, action_noise=action_noise, verbose=1, optimize_memory_usage=False, buffer_size=10000, gamma=1, 
             learning_rate=1e-3, learning_starts=0, train_freq=(1, 'episode'))
#model2 = TD3("MlpPolicy", env2, action_noise=action_noise, verbose=1, optimize_memory_usage=False, buffer_size=10000, gamma=1, 
             #learning_rate=1e-3, learning_starts=0, train_freq=(1, 'episode'))

#model2.learn(total_timesteps=10000, log_interval=1, reset_num_timesteps=False) 
#model2.save("attacker_TD3_random_defense_10000")
#model1.learn(total_timesteps=10000, log_interval=10, reset_num_timesteps=False) 
#model1.save("defender_TD3_random_attack_10000")

model1.save("forget_meta_defender_TD3_0000")
time.sleep(5)

for i in range(5):
    #model2.learn(total_timesteps=10000, log_interval=100, reset_num_timesteps=False) 
    #model2.save("ALTattacker_TD3_{}0000".format(i+1))
    model1.learn(total_timesteps=2000, log_interval=10, reset_num_timesteps=True)
    model1.save_replay_buffer("forget_TD3_replay_buffer_{}000".format(2*(i+1)))
    del model1
    model1=TD3.load("forget_meta_defender_TD3_{}000".format(2*(i)))
    print(f"The initial_model has {model1.replay_buffer.size()} transitions in its buffer")
    model1.load_replay_buffer("forget_TD3_replay_buffer_{}000".format(2*(i+1)))
    print(f"The loaded_model has {model1.replay_buffer.size()} transitions in its buffer")
    model1.set_env(env1)
    model1.learn(total_timesteps=2000, log_interval=10, reset_num_timesteps=True)
    print(f"The learned_model has {model1.replay_buffer.size()} transitions in its buffer")
    model1.save("forget_meta_defender_TD3_{}000".format(2*(i+1)))
    del model1
    model1=TD3.load("forget_meta_defender_TD3_{}000".format(2*(i+1)))
    model1.set_env(env1)


# remove to demonstrate saving and loading
del model1
#del model2

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -584     |
| time/              |          |
|    episodes        | 10       |
|    fps             | 102      |
|    time_elapsed    | 9        |
|    total_timesteps | 1000     |
| train/             |          |
|    actor_loss      | 4.21     |
|    critic_loss     | 237      |
|    learning_rate   | 0.001    |
|    n_updates       | 900      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -463     |
| time/              |          |
|    episodes        | 20       |
|    fps             | 97       |
|    time_elapsed    | 20       |
|    total_timesteps | 2000     |
| train/             |          |
|    actor_loss      | 8.01     |
|    critic_loss     

In [ ]:
# Pre-trainning Reptile Meta-Reinforcement Learning for Robust MTD

num_itr=10
num_task=20
alpha=1e-3
beta=1


# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

r=10*np.random.randint(11, size=num_config)
#print(r)
q=0.1*np.random.randint(11, size=num_config)
#print(q)
env = Defender_Env(r,q)

model = TD3("MlpPolicy", env, action_noise=action_noise, verbose=1, optimize_memory_usage=False, buffer_size=10000, gamma=1, 
             learning_rate= alpha, learning_starts=0, train_freq=(1, 'episode'))

model.save_replay_buffer("empty_replay_buffer")
meta_theta=model.policy.parameters_to_vector()

for i in range(num_itr):

    theta_lis=[]
    for _ in range(num_task):

        r=10*np.random.randint(11, size=num_config)
        print(r)
        q=0.1*np.random.randint(11, size=num_config)
        print(q)
        env = Defender_Env(r,q)
        model.set_env(env)
        model.policy.load_from_vector(meta_theta)
        for k in range(10):
            model.load_replay_buffer("empty_replay_buffer")
            #print(model.replay_buffer.size())
            model.learn(total_timesteps=100, log_interval=1, reset_num_timesteps=True)
        theta=model.policy.parameters_to_vector()
        #print(theta)
        theta_lis.append(theta)
    
    meta_theta=meta_theta-beta*(meta_theta-np.mean(theta_lis,axis=0))
    model.policy.load_from_vector(meta_theta)
    if (i+1)%2==0: model.save("meta_defender_TD3_{}".format(1000*(i+1)))

# remove to demonstrate saving and loading
del model

Streaming output truncated to the last 5000 lines.
|    ep_len_mean     | 100       |
|    ep_rew_mean     | -1.12e+03 |
| time/              |           |
|    episodes        | 1         |
|    fps             | 1175      |
|    time_elapsed    | 0         |
|    total_timesteps | 100       |
----------------------------------
[80 90 80 10]
[0.9 1.  0.1 0.1]
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 100       |
|    ep_rew_mean     | -1.36e+03 |
| time/              |           |
|    episodes        | 1         |
|    fps             | 1068      |
|    time_elapsed    | 0         |
|    total_timesteps | 100       |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 100       |
|    ep_rew_mean     | -2.68e+03 |
| time/              |           |
|    episodes        | 1         |


In [ ]:
class adapt_defender_Env(gym.Env):

    def __init__(self,r,q):
        
        self.rnd=0
        self.r=r
        self.q=q

        high = 1
        low = 0
        self.action_space = spaces.Box(
            low=low,
            high=high,
            shape=(num_config,),
            dtype=np.float32
        )

        #self.observation_space=spaces.MultiDiscrete([num_config, max_tau, max_tau])
        #self.observation_space=spaces.MultiDiscrete([num_config, 2])
        self.observation_space=spaces.Discrete(num_config)
        


    def step(self, action):
        
        
        self.rnd+=1
        #print(action)
        
        def_action=action/np.sum(action)
        #print(action)
        
        #att_config=np.argmax(def_action*q*r)
        
        #if self.training_length==0:
            #print('random attack!') 
            #att_action=np.array([1/num_config]*num_config)
        #else: 
            #att_action, _ = self.model.predict(self.obs)
            #att_action=att_action/np.sum(att_action)
        
        if self.mod==1:  
            att_action=np.array([1/num_config]*num_config)
            att_config=np.random.choice(num_config, p=att_action)
        
        else: att_config=np.argmax(def_action*self.q*self.r)
        
        
        old_config=self.config
        self.config = np.random.choice(num_config, p=def_action)
        #if self.config==old_config: self.tau+=1
        #else: 
            #self.tau=1
            #self.state=0
            #self.tau_a=0
        self.state=0
        if att_config==self.config and self.state==0: self.state=np.random.choice([0,1], p=[1-self.q[self.config], self.q[self.config]])
        #if self.state==1: self.tau_a+=1

        #if self.config==old_config: reward=-self.state*self.r[self.config]
        #else: reward=-M[old_config, self.config]
        reward=-self.state*self.r[self.config]-M[old_config, self.config]
        
        
        #print(self.state)

        
        done=False
        if self.rnd==max_rnd: done=True

        #self.obs=np.array([self.config,self.tau, self.tau_a])
        #self.obs=np.array([self.config,self.state])
        self.obs=self.config
        
        return self.obs, reward, done, {}

    def reset(self):

        self.rnd=0
        self.config=np.random.choice(num_config, p=initial_distribution)
        #self.state=0
        #self.tau=0
        #self.tau_a=0
        
        #self.training_length=110000
        #while os.path.exists("ALTattacker_TD3_"+str(self.training_length)+".zip")==False:
            #self.training_length-=10000
            #print(self.training_length)
            #if self.training_length==0: break
            #if os.path.exists("ALTattacker_TD3_"+str(self.training_length)+".zip"):
                #self.model = TD3.load("ALTattacker_TD3_"+str(self.training_length)+".zip")
        
        #self.mod=np.random.choice([0,1])
        self.mod=0
        #if self.mod<=2: self.r=r1
        #else: self.r=r2
        

        #self.obs=np.array([self.config,self.tau, self.tau_a])
        #self.obs=np.array([self.config,self.state])
        self.obs=self.config

        return self.obs

In [ ]:
#adapt meta-policies

q=np.array([0.2,0.1,0.7,0.5])
r1=np.array([40,50,10,25])
r2=np.array([80,0,60,0])

env = adapt_defender_Env(r1,q)

model = TD3.load("meta_defender_TD3_10000.zip")

meta_theta=model.policy.parameters_to_vector()
print(meta_theta)

# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = TD3("MlpPolicy", env, action_noise=action_noise, verbose=1, optimize_memory_usage=False, buffer_size=10000, gamma=1, 
             learning_rate=1e-2, learning_starts=0, train_freq=(1, 'episode'))

model.set_env(env)
model.policy.load_from_vector(meta_theta)

model.learn(total_timesteps=100, log_interval=10, reset_num_timesteps=False) 
model.save("adapt1_defender_TD3_100")
model.learn(total_timesteps=100, log_interval=10, reset_num_timesteps=False) 
model.save("adapt1_defender_TD3_200")
model.learn(total_timesteps=100, log_interval=10, reset_num_timesteps=False) 
model.save("adapt1_defender_TD3_300")
model.learn(total_timesteps=100, log_interval=10, reset_num_timesteps=False) 
model.save("adapt1_defender_TD3_400")

del model

[-0.28467244 -0.06103727 -0.19411764 ...  0.01631734 -0.27941808
 -0.24077067]
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -495     |
| time/              |          |
|    episodes        | 10       |
|    fps             | 134      |
|    time_elapsed    | 7        |
|    total_timesteps | 1000     |
| train/             |          |
|    actor_loss      | 522      |
|    critic_loss     | 6.87e+03 |
|    learning_rate   | 0.01     |
|    n_updates       | 900      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -499     |
| time/              |          |
|    episodes        | 20       |
|    fps             | 135      |
|

In [ ]:
#model1 = TD3.load("best_defender_TD3_random_attack_10000.zip")
#model1 = TD3.load("defender_TD3_argmax_attack_10000.zip")
#model1 = TD3.load("meta_defender_TD3_10000.zip")
model1=TD3.load("adapt1_defender_TD3_100.zip")
#model1=TD3.load("random_initial.zip")


# test policies for 1000 steps


initial_distribution=np.array([1,0,0,0])
q=np.array([0.2,0.1,0.7,0.5])
r1=np.array([40,50,10,25])
r2=np.array([80,0,60,0])
r=r1

rewards=[]
for _ in range(1000):
    reward=0
    config=np.random.choice(num_config, p=initial_distribution)
    rnd=0
    #state=0
    #tau=0
    #tau_a=0
    #obs=np.array([config,tau,tau_a])
    #obs=np.array([config,state])
    obs=config

    for _ in range(100):
        rnd+=1
        
        #RL attack
        #att_action, _ = model2.predict(obs)
        #att_action=att_action/np.sum(att_action)
        
        #random attack
        #att_action=np.array([1/num_config]*num_config)
        
        #att_action=np.random.uniform(low=0.0, high=1.0, size=4)
        #att_action=att_action/np.sum(att_action)

        #att_config=np.random.choice(num_config, p=att_action)

        #p=0.5
        #def_action=np.array([(1-p)/(num_config-1)]*num_config)
        #def_action[config]=p
        #def_action=np.array([1/num_config]*num_config)
        def_action, _ = model1.predict(obs)
        def_action=def_action/np.sum(def_action)
        #print(def_action)

        #myopic argmax attack
        #att_will_lis=def_action*q*r1
        #if state==1: att_will_lis[config]*=1/q[config]
        #att_config=np.argmax(att_will_lis)
        att_config=np.argmax(def_action*q*r)

        old_config=config
        config = np.random.choice(num_config, p=def_action)
        #state=0
        #if config==old_config: tau+=1
        #else: 
            #tau=1
            #state=0
            #tau_a=0
        
        state=0
        if att_config==config and state==0: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        #if att_config==config and config==old_config: state=1
        #if att_config==config and config!=old_config: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        #if state==1: tau_a+=1

        #reward+=state*r2[config]
        #else: reward+=M[old_config, config]
        reward+=state*r[config]+M[old_config, config]


        #obs=np.array([config,tau, tau_a])
        #obs=np.array([config,state])
        obs=config

    #print(reward)
    rewards.append(reward)

avg_reward=np.average(rewards)
print(avg_reward)

490.118


In [ ]:
#model1 = TD3.load("best_defender_TD3_random_attack_10000.zip")
#model1 = TD3.load("defender_TD3_argmax_attack_10000.zip")
#model1 = TD3.load("meta_defender_TD3_10000.zip")
model1=TD3.load("adapt1_defender_TD3_200.zip")
#model1=TD3.load("random_initial.zip")





initial_distribution=np.array([1,0,0,0])
q=np.array([0.2,0.1,0.7,0.5])
r1=np.array([40,50,10,25])
r2=np.array([80,0,60,0])
r=r1

rewards=[]
for _ in range(1000):
    reward=0
    config=np.random.choice(num_config, p=initial_distribution)
    rnd=0
    #state=0
    #tau=0
    #tau_a=0
    #obs=np.array([config,tau,tau_a])
    #obs=np.array([config,state])
    obs=config

    for _ in range(100):
        rnd+=1
        
        #RL attack
        #att_action, _ = model2.predict(obs)
        #att_action=att_action/np.sum(att_action)
        
        #random attack
        #att_action=np.array([1/num_config]*num_config)
        
        #att_action=np.random.uniform(low=0.0, high=1.0, size=4)
        #att_action=att_action/np.sum(att_action)

        #att_config=np.random.choice(num_config, p=att_action)

        #p=0.5
        #def_action=np.array([(1-p)/(num_config-1)]*num_config)
        #def_action[config]=p
        #def_action=np.array([1/num_config]*num_config)
        def_action, _ = model1.predict(obs)
        def_action=def_action/np.sum(def_action)
        #print(def_action)

        #myopic argmax attack
        #att_will_lis=def_action*q*r1
        #if state==1: att_will_lis[config]*=1/q[config]
        #att_config=np.argmax(att_will_lis)
        att_config=np.argmax(def_action*q*r)

        old_config=config
        config = np.random.choice(num_config, p=def_action)
        #state=0
        #if config==old_config: tau+=1
        #else: 
            #tau=1
            #state=0
            #tau_a=0
        
        state=0
        if att_config==config and state==0: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        #if att_config==config and config==old_config: state=1
        #if att_config==config and config!=old_config: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        #if state==1: tau_a+=1

        #reward+=state*r2[config]
        #else: reward+=M[old_config, config]
        reward+=state*r[config]+M[old_config, config]


        #obs=np.array([config,tau, tau_a])
        #obs=np.array([config,state])
        obs=config

    #print(reward)
    rewards.append(reward)

avg_reward=np.average(rewards)
print(avg_reward)

467.495


In [ ]:
#model1 = TD3.load("best_defender_TD3_random_attack_10000.zip")
#model1 = TD3.load("defender_TD3_argmax_attack_10000.zip")
#model1 = TD3.load("meta_defender_TD3_10000.zip")
model1=TD3.load("adapt1_defender_TD3_300.zip")
#model1=TD3.load("random_initial.zip")





initial_distribution=np.array([1,0,0,0])
q=np.array([0.2,0.1,0.7,0.5])
r1=np.array([40,50,10,25])
r2=np.array([80,0,60,0])
r=r1

rewards=[]
for _ in range(1000):
    reward=0
    config=np.random.choice(num_config, p=initial_distribution)
    rnd=0
    #state=0
    #tau=0
    #tau_a=0
    #obs=np.array([config,tau,tau_a])
    #obs=np.array([config,state])
    obs=config

    for _ in range(100):
        rnd+=1
        
        #RL attack
        #att_action, _ = model2.predict(obs)
        #att_action=att_action/np.sum(att_action)
        
        #random attack
        #att_action=np.array([1/num_config]*num_config)
        
        #att_action=np.random.uniform(low=0.0, high=1.0, size=4)
        #att_action=att_action/np.sum(att_action)

        #att_config=np.random.choice(num_config, p=att_action)

        #p=0.5
        #def_action=np.array([(1-p)/(num_config-1)]*num_config)
        #def_action[config]=p
        #def_action=np.array([1/num_config]*num_config)
        def_action, _ = model1.predict(obs)
        def_action=def_action/np.sum(def_action)
        #print(def_action)

        #myopic argmax attack
        #att_will_lis=def_action*q*r1
        #if state==1: att_will_lis[config]*=1/q[config]
        #att_config=np.argmax(att_will_lis)
        att_config=np.argmax(def_action*q*r)

        old_config=config
        config = np.random.choice(num_config, p=def_action)
        #state=0
        #if config==old_config: tau+=1
        #else: 
            #tau=1
            #state=0
            #tau_a=0
        
        state=0
        if att_config==config and state==0: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        #if att_config==config and config==old_config: state=1
        #if att_config==config and config!=old_config: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        #if state==1: tau_a+=1

        #reward+=state*r2[config]
        #else: reward+=M[old_config, config]
        reward+=state*r[config]+M[old_config, config]


        #obs=np.array([config,tau, tau_a])
        #obs=np.array([config,state])
        obs=config

    #print(reward)
    rewards.append(reward)

avg_reward=np.average(rewards)
print(avg_reward)

454.554


In [ ]:
#model1 = TD3.load("best_defender_TD3_random_attack_10000.zip")
#model1 = TD3.load("defender_TD3_argmax_attack_10000.zip")
#model1 = TD3.load("meta_defender_TD3_10000.zip")
model1=TD3.load("adapt1_defender_TD3_400.zip")
#model1=TD3.load("random_initial.zip")





initial_distribution=np.array([1,0,0,0])
q=np.array([0.2,0.1,0.7,0.5])
r1=np.array([40,50,10,25])
r2=np.array([80,0,60,0])
r=r1

rewards=[]
for _ in range(1000):
    reward=0
    config=np.random.choice(num_config, p=initial_distribution)
    rnd=0
    #state=0
    #tau=0
    #tau_a=0
    #obs=np.array([config,tau,tau_a])
    #obs=np.array([config,state])
    obs=config

    for _ in range(100):
        rnd+=1
        
        #RL attack
        #att_action, _ = model2.predict(obs)
        #att_action=att_action/np.sum(att_action)
        
        #random attack
        #att_action=np.array([1/num_config]*num_config)
        
        #att_action=np.random.uniform(low=0.0, high=1.0, size=4)
        #att_action=att_action/np.sum(att_action)

        #att_config=np.random.choice(num_config, p=att_action)

        #p=0.5
        #def_action=np.array([(1-p)/(num_config-1)]*num_config)
        #def_action[config]=p
        #def_action=np.array([1/num_config]*num_config)
        def_action, _ = model1.predict(obs)
        def_action=def_action/np.sum(def_action)
        #print(def_action)

        #myopic argmax attack
        #att_will_lis=def_action*q*r1
        #if state==1: att_will_lis[config]*=1/q[config]
        #att_config=np.argmax(att_will_lis)
        att_config=np.argmax(def_action*q*r)

        old_config=config
        config = np.random.choice(num_config, p=def_action)
        #state=0
        #if config==old_config: tau+=1
        #else: 
            #tau=1
            #state=0
            #tau_a=0
        
        state=0
        if att_config==config and state==0: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        #if att_config==config and config==old_config: state=1
        #if att_config==config and config!=old_config: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        #if state==1: tau_a+=1

        #reward+=state*r2[config]
        #else: reward+=M[old_config, config]
        reward+=state*r[config]+M[old_config, config]


        #obs=np.array([config,tau, tau_a])
        #obs=np.array([config,state])
        obs=config

    #print(reward)
    rewards.append(reward)

avg_reward=np.average(rewards)
print(avg_reward)

486.824


In [ ]:
#model2 = TD3.load("attacker_TD3_policy_20000.zip")

      
for i in range(10):
  model1 = TD3.load("ALTdefender_TD3_{}0000.zip".format(i+1))
  rewards=[]
  for _ in range(1000):
    reward=0
    config=np.random.choice(num_config, p=initial_distribution)
    rnd=0
    state=0
    tau=0
    tau_a=0
    #obs=np.array([config,tau,tau_a])
    obs=np.array([config,state])

    
    for _ in range(100):
        rnd+=1
        
        #RL attack
        #att_action, _ = model2.predict(obs)  
        #att_action=att_action/np.sum(att_action)
        
        #random attack
        #att_action=np.array([1/num_config]*num_config)
        #att_action=np.array([1.0,0.0,0.0,0.0])
        #att_action=np.random.uniform(low=0.0, high=1.0, size=4)
        #att_action=att_action/np.sum(att_action)

        #att_config=np.random.choice(num_config, p=att_action)
        #print(att_config)


        #constant defense
        #def_action=np.array([1/num_config]*num_config)
        #def_action=np.array([1.0,0.0,0.0,0.0])
        #def_action=np.array([0.0,0.0,1.0,0.0])
        
        #random defense
        #def_action=np.random.uniform(low=0.0, high=1.0, size=4)
        #def_action=def_action/np.sum(def_action)
       
        #RL defense
        def_action, _ = model1.predict(obs)
        def_action=def_action/np.sum(def_action)

        #myopic argmax attack
        att_config=np.argmax(def_action*q*r)
        #att_config=np.argmin(def_action*q*r)
        #print(att_config)


        old_config=config
        config = np.random.choice(num_config, p=def_action)

        if config==old_config: tau+=1
        else: 
            tau=1
            state=0
            tau_a=0
        
        if att_config==config and state==0: state=np.random.choice([0,1], p=[1-q[config], q[config]])
        if state==1: tau_a+=1

        #reward+=state*r[config]
        if config==old_config: reward+=state*r[config]
        else: reward+=M[old_config, config]

        #obs=np.array([config,tau, tau_a])
        obs=np.array([config,state])

    #print(reward)
    rewards.append(reward)

  avg_reward=np.average(rewards)
  print(avg_reward)

142.224
316.912
264.973
143.88
196.926
233.153
229.502
200.475
199.596
240.903
